In [1]:
import os
from importlib import reload

import gpr
import cfGPR
import vonkarman as vk
import plotGPR

import gbutil

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

# ToDo

1. Check that correlation functions are positive definite

2. Clean up DESworkspace directory

3. Use ckdtree to get all pairs to quickly get the average xiplus we want. Be sure to go out a bit farther separation than before. Make sure to use maxErr somehow. To not have too much noise.

4. Optimize kernel with respect to this xiplus average

5. Use the von Karman covariance function to make simulated data (draw from a Guassian with a von Karman covariance function with known parameters). Use this to check that it is doing what we want it to do.

6. Try just flipping the sign of the uv term in the covariance matrix because, who knows, maybe somewhere along the way we made a mistake.

7. For some exposure, look at the u and v kernel from the 2 Kernel method and compare them to the Cuu and Cuv kernels from the von Karman method

8. Use the von Karman covariance function to make a 2 Kernel model. Use the Cuu part to make a kernel for u, and use the Cvv part to make a kernel for v.

9. Two Layers: Fit a von karman with a big window (30 arcmin or so?); this should pay attention to the outer structure. Subtract that from the correlation function (that you just fitted). Then, fit the residuals with another von Karman (on a smaller scale (5 arcmin or so). When applying this to the GPR algorithm, K_big + K_small and Ks_big and Ks_small should just add together since they should be uncorrelated if they are from different turbulent layers.

10. write vKcfGPR.py

11. write a separate script that just has the functions for opening, sigma clipping, that sort of thing. Maybe also separate out the functions for synthetic data generation. Also separate the functions for saving and loading from npz?

12. Code in an alert if an optimizer seemed to get stuck at a bound


# plotGPR.py

1. Look at moving the titles on AstrometricError and DivCurl plots

2. Add more optional info onto the plot. Exposure number, number of data points. For 2d plots, convert from bins to angle separation

3. Write code to make nice comparison plots to compare two different methods

4. Finish commenting plotGPR.py

5. Move calc functions into a new script, utils.py

6. Consider specifying some rcparams at the top of plotGPR.py

In [23]:
# reload(plotGPR)

# exposures = np.arange(450, 500)
# for exposure in exposures:
#     GP = gpr.GPR('dxdy', npz=f"../exposures/{exposure}/{exposure}.npz")
    
#     x = GP.Xtest[:, 0]*u.deg
#     y = GP.Xtest[:, 1]*u.deg
#     dx = GP.Ytest[:, 0]*u.mas
#     dy = GP.Ytest[:, 1]*u.mas
#     err = GP.Etest*u.mas
#     x2 = GP.Xtest[:, 0]*u.deg
#     y2 = GP.Xtest[:, 1]*u.deg
#     dx2 = GP.Ytest[:, 0]*u.mas - GP.fbar_s[:, 0]*u.mas
#     dy2 = GP.Ytest[:, 1]*u.mas - GP.fbar_s[:, 1]*u.mas
#     err2 = GP.Etest*u.mas
    
#     plotGPR.AstrometricResiduals(
#         x, y, dx, dy, err,
#         x2=x2, y2=y2, dx2=dx2, dy2=dy2, err2=err2,
#         savePath=f"../exposures/{exposure}/",
#         saveExt=f"{exposure}",
#         plotShow=False)

#     plotGPR.DivCurl(
#         x, y, dx, dy, err,
#         x2=x2, y2=y2, dx2=dx2, dy2=dy2, err2=err2,
#         savePath=f"../exposures/{exposure}/",
#         saveExt=f"{exposure}",
#         plotShow=False)

#     plotGPR.Correlation(
#         x, y, dx, dy,
#         x2=x2, y2=y2, dx2=dx2, dy2=dy2,
#         savePath=f"../exposures/{exposure}/",
#         saveExt=f"{exposure}",
#         printFile="corr.out",
#         plotShow=False)

#     plotGPR.Correlation2D(
#         x, y, dx, dy,
#         x2=x2, y2=y2, dx2=dx2, dy2=dy2,
#         savePath=f"../exposures/{exposure}/",
#         saveExt=f"{exposure}",
#         plotShow=False)